In [13]:
%pip install --upgrade pip
%pip cache purge

# General
import pandas as pd
import numpy as np
import io
import os
from PIL import Image

# Neural network (mobilenet)
%pip install tensorflow
%pip install tensorflow-gpu
import tensorflow as tf
print('\n', 'Tensorflow version ' + tf.__version__, '\n')
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

# map reduce
# %pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

# Acces bucket
from google.cloud import storage

# PCA
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT

# Visu
import pprint
import matplotlib.pyplot as plt

print('\n', 'Done')


Note: you may need to restart the kernel to use updated packages.
Files removed: 125
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requireme

In [2]:
# test

def list_folders(bucket_name, prefix):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)  # Filter blobs by prefix

    folders = set()
    for blob in blobs:
        folder = blob.name.split('/')[1]  # Get the folder name inside Test1
        if folder:
            folders.add(folder)

    return folders


bucket_name = 'data-fruits-p8'
prefix = 'Test1/'
folders = list_folders(bucket_name, prefix)

print(f'Number of folders on Google Cloud Storage: {len(folders)}', '\n') # 131

print("15 first folders in bucket:", '\n')
for i, folder in enumerate(list(folders)[:10]):
    print(f'{i+1}   {folder}')


Number of folders on Google Cloud Storage: 131 

15 first folders in bucket: 

1   Quince
2   Mulberry
3   Kaki
4   Tomato Maroon
5   Plum
6   Tomato not Ripened
7   Cherry Wax Red
8   Huckleberry
9   Apple Red 1
10   Avocado ripe


In [3]:
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

sc = spark.sparkContext

spark


24/05/12 15:21:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
images = spark.read.format("binaryFile") \
    .option("pathGlobFilter", "*.jpg") \
    .option("recursiveFileLookup", "true") \
    .load("gs://data-fruits-p8/Test1")

type(images)


pyspark.sql.dataframe.DataFrame

In [5]:
images.show(5, True)


+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|gs://data-fruits-...|2024-05-12 13:22:...|  7353|[FF D8 FF E0 00 1...|
|gs://data-fruits-...|2024-05-12 13:22:...|  7350|[FF D8 FF E0 00 1...|
|gs://data-fruits-...|2024-05-12 13:22:...|  7349|[FF D8 FF E0 00 1...|
|gs://data-fruits-...|2024-05-12 13:22:...|  7348|[FF D8 FF E0 00 1...|
|gs://data-fruits-...|2024-05-12 13:22:...|  7328|[FF D8 FF E0 00 1...|
+--------------------+--------------------+------+--------------------+
only showing top 5 rows



In [6]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))


root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------+----------+
|path                                              |label     |
+--------------------------------------------------+----------+
|gs://data-fruits-p8/Test1/Watermelon/r_106_100.jpg|Watermelon|
|gs://data-fruits-p8/Test1/Watermelon/r_109_100.jpg|Watermelon|
|gs://data-fruits-p8/Test1/Watermelon/r_108_100.jpg|Watermelon|
|gs://data-fruits-p8/Test1/Watermelon/r_107_100.jpg|Watermelon|
|gs://data-fruits-p8/Test1/Watermelon/r_95_100.jpg |Watermelon|
+--------------------------------------------------+----------+
only showing top 5 rows

None


In [7]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

# new_model.summary()


In [8]:
brodcast_weights = sc.broadcast(new_model.get_weights())


In [9]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)

    return new_model


In [10]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)

    return preprocess_input(arr)


def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input) 

    return pd.Series(preds)


@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER) # type hints instead of type ?
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for i, content_series in enumerate(content_series_iter):
        print('\n', i)
        yield featurize_series(model, content_series)


/usr/lib/spark/python/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [11]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

features_df.persist()


DataFrame[path: string, label: string, features: array<float>]

In [12]:
PATH_extraction = "gs://data-fruits-p8/feature-extraction"

features_df.write.mode("overwrite").parquet(PATH_extraction)


2024-05-12 15:24:36.533688: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:24:36.533692: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:24:36.536255: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:24:36.536667: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:24:36.542595: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:24:36.542595: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:24:36.543668: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not b

24/05/12 15:26:23 WARN BlockManager: Putting block rdd_21_7 failed due to exception org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_1/3857089266.py", line 36, in featurize_udf
  File "/tmp/ipykernel_1/3857089266.py", line 19, in featurize_series
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/series.py", line 584, in __init__
    data = sanitize_array(data, index, dtype, copy)
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/construction.py", line 659, in sanitize_array
    subarr = _sanitize_ndim(subarr, data, dtype, index, allow_2d=allow_2d)
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/construction.py", line 718, in _sanitize_ndim
    raise ValueError(
ValueError: Data must be 1-dimensional, got ndarray of shape (1165, 1280) instead
.
24/05/12 15:26:23 WARN BlockManager: Putting block rdd_21_4 failed due to exception org.apache.spark.api.python.PythonException: Traceback (most recent call las

37/37 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step6) / 20]
37/37 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step


24/05/12 15:26:24 WARN BlockManager: Putting block rdd_21_5 failed due to exception org.apache.spark.TaskKilledException.
24/05/12 15:26:24 WARN BlockManager: Block rdd_21_5 could not be removed as it was not found on disk or in memory
24/05/12 15:26:24 WARN TaskSetManager: Lost task 5.0 in stage 4.0 (TID 847) (5024d3e999d0 executor driver): TaskKilled (Stage cancelled)
24/05/12 15:26:24 WARN BlockManager: Putting block rdd_21_6 failed due to exception org.apache.spark.TaskKilledException.
24/05/12 15:26:24 WARN BlockManager: Block rdd_21_6 could not be removed as it was not found on disk or in memory
24/05/12 15:26:24 WARN TaskSetManager: Lost task 6.0 in stage 4.0 (TID 848) (5024d3e999d0 executor driver): TaskKilled (Stage cancelled)


2024-05-12 15:26:25.003020: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:26:25.007860: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:26:25.030101: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:26:25.034490: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 15:26:25.070040: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 15:26:25.095084: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optim

Py4JJavaError: An error occurred while calling o93.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:650)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:309)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.

In [ ]:
df = pd.read_parquet(PATH_extraction, engine='pyarrow')
type(df)


In [ ]:
df.head()


In [ ]:
# Define a UDF to convert array-like objects to DenseVectors
array_to_dense_vector_udf = udf(lambda arr: Vectors.dense(arr), VectorUDT())

# Apply the UDF to the 'features' column
vectors_df = features_df.withColumn('vectors', array_to_dense_vector_udf(col('features')))

vectors_df.show()
vectors_df.printSchema()


In [ ]:
# Perform PCA
pca = PCA(k=180, inputCol="vectors", outputCol="pca_features") # environ 0.9 de variance expliquée
model_pca = pca.fit(vectors_df)
result_pca = model_pca.transform(vectors_df)

# Output result
result_pca.show(truncate=False)


In [ ]:
print(model_pca.explainedVariance)

# Plot
explained_variance_ratios = model_pca.explainedVariance.toArray()

# Compute cumulative explained variance
cumulative_variance = np.cumsum(explained_variance_ratios)

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-')
plt.title('Cumulative Variance Explained by PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance Explained')
plt.grid(True)
plt.show()


In [ ]:
PATH_Result_PCA = "gs://data-fruits-p8/pca"

# delete feature extraction col & vectors (étapes intermédiaires, plus nécessaire.)
result_pca = result_pca.drop('features', 'vectors')
result_pca.show()

result_pca.write.mode("overwrite").parquet(PATH_Result_PCA)


2024-05-12 15:26:26.575852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-12 15:26:26.605899: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


24/05/12 15:26:26 WARN ArrowPythonRunner: Incomplete task 2.0 in stage 4 (TID 844) interrupted: Attempting to kill Python Worker
24/05/12 15:26:26 WARN BlockManager: Putting block rdd_21_2 failed due to exception org.apache.spark.TaskKilledException.
24/05/12 15:26:26 WARN BlockManager: Block rdd_21_2 could not be removed as it was not found on disk or in memory
24/05/12 15:26:26 WARN ArrowPythonRunner: Incomplete task 0.0 in stage 4 (TID 842) interrupted: Attempting to kill Python Worker
24/05/12 15:26:26 WARN BlockManager: Putting block rdd_21_0 failed due to exception org.apache.spark.TaskKilledException.
24/05/12 15:26:26 WARN BlockManager: Block rdd_21_0 could not be removed as it was not found on disk or in memory
24/05/12 15:26:26 WARN TaskSetManager: Lost task 0.0 in stage 4.0 (TID 842) (5024d3e999d0 executor driver): TaskKilled (Stage cancelled)
24/05/12 15:26:26 WARN TaskSetManager: Lost task 2.0 in stage 4.0 (TID 844) (5024d3e999d0 executor driver): TaskKilled (Stage cancell